In [288]:
from tqdm import tqdm
from termcolor import colored
import sys
sys.path.append("../../")

from process_data.datasets import DATASETS

In [21]:
DATASETS

{'RTC': PosixPath('/home/itmo/datasets/RTC'),
 'ted': PosixPath('/home/itmo/datasets/ted'),
 'mix_test': PosixPath('/home/itmo/datasets/mix_test'),
 'RTC_subset': PosixPath('/home/itmo/datasets/RTC_subset'),
 'statmt': PosixPath('/home/itmo/datasets/statmt'),
 'wikimatrix': PosixPath('/home/itmo/datasets/wikimatrix'),
 'tmp': PosixPath('/home/itmo/datasets/tmp'),
 'ntrex': PosixPath('/home/itmo/datasets/ntrex'),
 'nu': PosixPath('/home/itmo/datasets/nu'),
 'kaznu': PosixPath('/home/itmo/datasets/kaznu'),
 'opus': PosixPath('/home/itmo/datasets/opus')}

In [28]:
alignment_path = DATASETS['tmp'] / 'gdf.align'

alignments = []
with open(alignment_path) as f:
    for line in tqdm(f):
        alignments.append(line.rstrip('\n'))

16048810it [00:05, 2816406.37it/s]


In [29]:
alignments[0]

'0-0 1-2 2-3 3-1 4-4 5-5 6-10 7-11 8-12 9-9 10-7 11-6 12-13'

In [30]:
len(alignments)

16048810

In [15]:
def read_data(data_path, src_lang='kk', tgt_lang='ru'):
        src_data = []
        tgt_data = []
        with open(str(data_path) + "." + src_lang) as f, open(str(data_path) + "." + tgt_lang) as g:
            for src_line, tgt_line in zip(f, g):
                if src_line != "" and tgt_line != "":
                    src_data.append(src_line.replace("\n", ""))
                    tgt_data.append(tgt_line.replace("\n", ""))
        return src_data, tgt_data

In [16]:
train_paths = [DATASETS['nu'], DATASETS['kaznu'], DATASETS['opus'], DATASETS['RTC'],  DATASETS['statmt'],  DATASETS['wikimatrix']] 
train_paths = [p /  'processed' / 'train' / 'kk-ru_processed' for p in train_paths]
train_data = [read_data(p) for p in tqdm(train_paths)]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:21<00:00,  3.53s/it]


In [31]:
train_data_lengths = [len(src_data) for src_data, tgt_data in train_data]
train_data_lengths 

[860421, 80627, 2177833, 12752816, 13460, 163653]

In [32]:
sum(train_data_lengths)

16048810

In [17]:
train_data_indexes = [860421, 941048, 3118881, 15871697, 15885157, 16048810]

In [36]:
train_data_alignments = []
for i, end_index in enumerate(train_data_indexes):
    if i == 0:
        train_data_alignments.append(alignments[ : end_index])
    else:
        train_data_alignments.append(alignments[train_data_indexes[i - 1] : end_index])

In [37]:
train_data_alignments_length = [len(a) for a in train_data_alignments]
train_data_alignments_length

[860421, 80627, 2177833, 12752816, 13460, 163653]

In [257]:
i = 0
j = 206
example = train_data_alignments[i][j].split()
example_src_text, example_tgt_text = train_data[i][0][j].split(), train_data[i][1][j].split()

print(len(example_src_text))
print(" ".join(example_src_text), "\n")

print(len(example_tgt_text))
print(" ".join(example_tgt_text))
      
example_splitted = [x.split("-") for x in example]
example_splitted

11
Салтанатты ашылу рәсімінде филармонияның би ұжымы қазақ ұлттық биін биледі . 

14
Торжественный момент открытия был ознаменован замечательным национальным танцем в исполнении танцевального коллектива филармонии .


[['0', '0'],
 ['1', '1'],
 ['1', '2'],
 ['2', '1'],
 ['3', '3'],
 ['3', '4'],
 ['3', '5'],
 ['4', '3'],
 ['5', '3'],
 ['7', '6'],
 ['8', '7'],
 ['9', '8'],
 ['10', '13'],
 ['13', '10']]

In [269]:
src2tgt_units = {}
tgt2src_units = {}


for i, (src_index, tgt_index) in enumerate(example_splitted):
    src_index, tgt_index = int(src_index), int(tgt_index)
    if src_index < len(example_src_text) and tgt_index < len(example_tgt_text):
        # print(example_src_text[src_index], "<->", example_tgt_text[tgt_index])
        if src_index in src2tgt_units:
            src2tgt_units[src_index] += [tgt_index]
        else:
            src2tgt_units[src_index] = [tgt_index]
            
        if tgt_index in tgt2src_units:
            tgt2src_units[tgt_index] += [src_index]
        else:
            tgt2src_units[tgt_index] = [src_index]

units = src2tgt_units.copy()
for src_index, tgt_indexes in src2tgt_units.items():
    if len(tgt_indexes) > 1:
        for tgt_index in tgt_indexes:
            if tgt_index != src_index and tgt_index in units and len(units[tgt_index]) == 1:
                units.pop(tgt_index)
print(src2tgt_units)

for src_index, tgt_indexes in units.items():
    print(src_index, "<->", tgt_indexes)
    print(f"{example_src_text[src_index]} <- {' '.join([example_tgt_text[tgt_index] for tgt_index in tgt_indexes])}")
    print(f"{example_tgt_text[src_index]} -> {' '.join([example_src_text[src_index] if src_index < len(example_src_text) else '' for src_index in tgt_indexes])}\n")

{0: [0], 1: [1, 2], 2: [1], 3: [3, 4, 5], 4: [3], 5: [3], 7: [6], 8: [7], 9: [8], 10: [13]}
0 <-> [0]
Салтанатты <- Торжественный
Торжественный -> Салтанатты

1 <-> [1, 2]
ашылу <- момент открытия
момент -> ашылу рәсімінде

3 <-> [3, 4, 5]
филармонияның <- был ознаменован замечательным
был -> филармонияның би ұжымы

7 <-> [6]
ұлттық <- национальным
танцем -> қазақ

8 <-> [7]
биін <- танцем
в -> ұлттық

9 <-> [8]
биледі <- в
исполнении -> биін

10 <-> [13]
. <- .
танцевального -> 



In [338]:
import random

def generate_csw(alignment, src_line, tgt_line, percent=15, color=False, print_alignment=False):
    alignment = [x.split("-") for x in alignment.split()]
    
    src_line = src_line.split()
    tgt_line = tgt_line.split()
    
    src2tgt_units = {}
    for i, (src_index, tgt_index) in enumerate(alignment):
        src_index, tgt_index = int(src_index), int(tgt_index)
        if src_index < len(example_src_text) and tgt_index < len(example_tgt_text):
            if src_index in src2tgt_units:
                src2tgt_units[src_index] += [tgt_index]
            else:
                src2tgt_units[src_index] = [tgt_index]
            
    units = src2tgt_units.copy()
    for src_index, tgt_indexes in src2tgt_units.items():
        if len(tgt_indexes) > 1:
            for tgt_index in tgt_indexes:
                if tgt_index != src_index and tgt_index in units and len(units[tgt_index]) == 1:
                    units.pop(tgt_index)
                    
    num_replacements = int(len(src_line) * percent / 100) if len(src_line) >= 7 else 1
    src_indexes = list(units.keys())
    random.shuffle(src_indexes)

    for src_index in src_indexes[:num_replacements]:
        tgt_indexes = units[src_index]
        
        src2tgt_alignment = random.random() > 0.5

        if src2tgt_alignment:
            if src_index >= len(src_line):
                continue
            replacement = ' '.join([tgt_line[tgt_index] for tgt_index in tgt_indexes if tgt_index < len(tgt_line)])
            if print_alignment:
                print(f"{src_line[src_index]} -> {replacement}")
            src_line[src_index] = colored(replacement, 'green') if color else replacement
        else:
            replacement = tgt_line[src_index] if src_index < len(tgt_line) else ""
            if print_alignment:
                src_replacement = ' '.join([src_line[tgt_index] for tgt_index in tgt_indexes if tgt_index < len(src_line)])
                print(f"{src_replacement} <- {replacement}")

            if tgt_indexes[0] >= len(src_line):
                continue
            src_line[tgt_indexes[0]] =  colored(replacement, 'green') if color else replacement
        
            for src_index in tgt_indexes[1:]:
                if src_index < len(src_line): 
                    src_line.pop(src_index)
    return " ".join(src_line)

In [327]:
csw_train_datasets = []

for i, (src_dataset, tgt_dataset) in enumerate(train_data):
    csw_dataset = []
    alignment_dataset = train_data_alignments[i]
    for src_line, tgt_line, alignment in tqdm(zip(src_dataset, tgt_dataset, alignment_dataset)):
        csw_line = generate_csw(alignment, src_line, tgt_line)
        csw_dataset.append(csw_line)
    csw_train_datasets.append(csw_dataset)
        

860421it [00:20, 41640.30it/s]
80627it [00:01, 40738.83it/s]
2177833it [00:18, 117852.15it/s]
12752816it [02:29, 85296.07it/s]
13460it [00:00, 45538.08it/s]
163653it [00:02, 75074.11it/s]


In [336]:
for i, path in enumerate(train_paths):
    csw_dataset = csw_train_datasets[i]
    save_path = str(path).replace('kk-ru_processed', 'cs-4-ru_processed')
    with open(save_path + ".kk", 'w') as f_src, open(save_path + ".ru", 'w') as f_tgt:
        for src_line, tgt_line in tqdm(zip(csw_dataset, train_data[i][1])):
            f_src.write(src_line.replace("\n", "") + "\n")
            f_tgt.write(tgt_line.replace("\n", "") + "\n")

860421it [00:02, 380274.29it/s]
80627it [00:00, 356148.24it/s]
2177833it [00:02, 763401.76it/s]
12752816it [00:18, 691093.76it/s]
13460it [00:00, 339648.36it/s]
163653it [00:00, 563137.40it/s]


In [334]:
len(csw_train_datasets[0])

860421

In [335]:
len(train_data[0][0])

860421

In [341]:
from random import randrange

i = 0
j = 301

for j in  random.sample(range(1, 50000), 30):
    example = train_data_alignments[i][j]
    example_src_text, example_tgt_text = train_data[i][0][j], train_data[i][1][j]
    
    csw_example = generate_csw(example, example_src_text, example_tgt_text, color=True, print_alignment=True)
    
    
    print(example_src_text)
    print(csw_example)
    print(example_tgt_text, "\n")

біраз -> проделала
- 2005 жылдан бастап бұл ұйым біраз жол жүріп өтті .
- 2005 жылдан бастап бұл ұйым проделала жол жүріп өтті .
- С 2005 года данная организация проделала очень большой путь . 

иммунитет -> до
сақталады -> сохраняется
Вакциналарға иммунитет тудыру үшін 2 күннен 7-15 күнге дейін уақыт керек , иммунитет 6-12 айға сақталады .
Вакциналарға иммунитет тудыру үшін 2 күннен 7-15 күнге дейін уақыт керек , до 6-12 айға сохраняется .
Выработка иммунитета на введение вакцин составляет от 2 до 7–15 дней и сохраняется от 6 до 12 месяцев . 

Ресей -> 
Президент Нұрсұлтан Назарбаев Ресей Ғылым академиясы экономика институтының Құрмет белгісімен марапатталды
Президент Нұрсұлтан Назарбаев  Ғылым академиясы экономика институтының Құрмет белгісімен марапатталды
Институт экономики РАН наградил Президента Нурсултана Назарбаева Почетным знаком 

Нұр <- противодействию
мүшелері -> члены
 <- коррупции
басшылары -> руководители
Кеңес отырысына « Нұр Отан » партиясы облыстық филиалы төрағасының

In [ ]:
53
100

In [237]:
src2tgt_units

{0: [0],
 1: [12],
 2: [13],
 3: [14],
 4: [15],
 5: [11],
 6: [10],
 7: [8],
 8: [9],
 9: [16],
 10: [17],
 11: [5],
 12: [1],
 13: [2],
 14: [3],
 15: [4],
 17: [32],
 18: [33],
 19: [31],
 20: [28],
 21: [27],
 22: [22],
 23: [24],
 24: [25],
 26: [23],
 27: [20],
 28: [21],
 29: [18],
 30: [34]}

In [214]:
tgt2src_units

{0: [0],
 3: [1, 2],
 4: [1],
 5: [2],
 1: [3, 4],
 2: [3, 5],
 9: [6],
 10: [6],
 11: [6],
 8: [7],
 7: [8],
 6: [9, 10, 11],
 22: [14],
 17: [16],
 16: [17],
 23: [18]}

In [174]:
for i, alignment in enumerate(example_splitted):
    src_index, tgt_index = int(alignment[0]), int(alignment[1])
    if src_index < len(example_src_text):
        print(example_src_text[src_index], "->", example_tgt_text[tgt_index])

Қазақстан -> Казахстан
әлемдегі -> в
дамыған -> наиболее
отыз -> тридцати
елдің -> стран
қатарына -> число
үшін -> чтобы
жағдай -> любых
болса -> при
құрылымдық -> Структурные
реформалар -> реформы
жалғаса -> продолжаться
тиіс -> должны
» -> »
, -> ,
- -> -
деді -> развитых
Қ.Тоқаев -> подчеркнул
, -> ,


IndexError: list index out of range